In [2]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 1
%aimport VQE, GroupingAlgorithm, HEEM_VQE_Functions, utils

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from VQE import VQE
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from utils import *
from qiskit import IBMQ
from qiskit import Aer
from qiskit.circuit.library import EfficientSU2
from qiskit.utils.quantum_instance import QuantumInstance
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.test.mock import FakeVigo
import os
from joblib import Parallel, delayed
from IPython.display import display, clear_output
from GroupingAlgorithm import grouping_shuffle
from tqdm.notebook import tqdm_notebook

os.environ['QISKIT_IN_PARALLEL'] = 'TRUE'

In [4]:
import contextlib
import joblib

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)

        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()   

In [5]:
def molecule(d, quantum_instance, conectivity, basis):
    print(d)
    E = np.zeros(4)

    def callback( evals, params ):  
        display("{}, {}".format(len(evaluations),evals))
        clear_output(wait=True)
        parameters.append(params)
        evaluations.append(evals)

    parameters  = []
    evaluations = []


    qubit_op, init_state = LiH(d, initial_state=True)
#     qubit_op = grouping_shuffle(qubit_op, basis, coupling_map)
    
    num_qubits = qubit_op.num_qubits

    # Exact energy
    E[0] = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op).eigenvalue.real
    print('Exact energy: {}'.format(E[0]))

    optimizer = SPSA(maxiter=2, last_avg=1)
    ansatz = init_state.compose(EfficientSU2(num_qubits, ['ry', 'rz'], entanglement='linear', reps=1))
    num_var = ansatz.num_parameters
    initial_params = np.random.rand(num_var)


    solver_TPB = VQE(ansatz, optimizer, initial_params, grouping='TPB', quantum_instance=quantum_instance,
                     callback=callback, order=basis)
    solver_ENT = VQE(ansatz, optimizer, initial_params, grouping='Entangled', quantum_instance=quantum_instance,
                     callback=callback, order=basis)
    solver_HEEM = VQE(ansatz, optimizer, initial_params, grouping='Entangled', conectivity=conectivity,
                      quantum_instance=quantum_instance, callback=callback, order=basis)


#     print('-' * 100)
#     print('Computing TPB:')
#     E[1] = solver_TPB.compute_minimum_eigenvalue(qubit_op).eigenvalue.real

#     print('-' * 100)
#     print('Computing ENT:')
#     E[2] = solver_ENT.compute_minimum_eigenvalue(qubit_op).eigenvalue.real

#     print('-' * 100)
    print('Computing HEEM:')
#     E[3] = solver_HEEM.compute_minimum_eigenvalue(qubit_op).eigenvalue.real

    return E

In [27]:
def molecule_potentials_comparison(distances, quantum_instance, conectivity, basis, file_name_out):
    E_EXACT = np.zeros_like(distances)
    E_TPB = np.zeros_like(distances)
    E_ENT = np.zeros_like(distances)
    E_HEEM = np.zeros_like(distances)


    with tqdm_joblib(tqdm_notebook(desc="Computing", total=len(distances))) as progress_bar:
        results = Parallel(n_jobs=24)(delayed(molecule)(d, quantum_instance, conectivity, basis) for d in distances)
        
    for i in range(len(distances)):
        E_EXACT[i] = results[i][0]
        E_TPB[i] = results[i][1]
        E_ENT[i] = results[i][2]
        E_HEEM[i] = results[i][3]

    # Save results
    np.savez(file_name_out, distances=distances, E_EXACT=E_EXACT, E_TPB=E_TPB, E_ENT=E_ENT, E_HEEM=E_HEEM)

    return E_EXACT, E_TPB, E_ENT, E_HEEM

In [21]:
distances = np.linspace(0.5, 5, 100)
backend = Aer.get_backend('qasm_simulator')
device_backend = FakeVigo()
device = QasmSimulator.from_backend(device_backend)
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)
basis_gates = noise_model.basis_gates
NUM_SHOTS = 2 ** 13  # Number of shots for each circuit
qi = QuantumInstance(backend=backend, coupling_map=coupling_map, noise_model=noise_model, basis_gates=basis_gates, shots=NUM_SHOTS)
basis = [4, 6, 7, 8, 9, 5, 3, 2, 1]

In [22]:
file_name_out = '../data/LiH_' + qi.backend_name + '_NUM_SHOTS=' + str(NUM_SHOTS) + '_dist=' + str(distances[0]) + '_to_' + str(distances[-1])

In [23]:
molecule(1, qi, coupling_map, basis)

1
Exact energy: -1.1724576315066804
Computing HEEM:


array([-1.17245763,  0.        ,  0.        ,  0.        ])

In [28]:
test = molecule_potentials_comparison(distances, qi, coupling_map, basis, file_name_out)

Computing:   0%|          | 0/100 [00:00<?, ?it/s]

[-0.97926755 -1.01155042 -1.04316568 -1.07233236 -1.09804629 -1.11981176
 -1.13748502 -1.15116797 -1.16112616 -1.16772245 -1.17136308 -1.17245763
 -1.17139226 -1.16851501 -1.16413024 -1.15849903 -1.15184288 -1.14434889
 -1.13617491 -1.12745441 -1.11830058 -1.10880972 -1.09906409 -1.08913418
 -1.07908058 -1.06895555 -1.05880426 -1.04866588 -1.03857445 -1.02855963
 -1.0186473  -1.00886015 -0.99921805 -0.98973853 -0.98043701 -0.97132715
 -0.96242104 -0.95372943 -0.94526183 -0.93702666 -0.92903128 -0.92128206
 -0.91378432 -0.90654236 -0.89955936 -0.89283731 -0.88637696 -0.88017775
 -0.87423775 -0.8685537  -0.86312099 -0.85793378 -0.8529851  -0.84826699
 -0.8437707  -0.83948685 -0.83540564 -0.83151704 -0.82781093 -0.82427727
 -0.82090619 -0.81768814 -0.8146139  -0.8116747  -0.8088622  -0.8061685
 -0.80358621 -0.80110838 -0.79872852 -0.79644059 -0.79423894 -0.79211834
 -0.79007391 -0.78810114 -0.78619583 -0.78435408 -0.78257228 -0.78084707
 -0.77917533 -0.77755416 -0.77598088 -0.77445299 -0.

In [ ]:
data = np.load(file_name_out + '.npz')

E_EXACT = data['E_EXACT']
E_TPB = data['E_TPB']
E_ENT = data['E_ENT']
E_HEEM = data['E_HEEM']

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 4.5))
ax.set_xlabel(r'$d$')
ax.set_ylabel(r'$E$')

ax.plot(distances, E_TPB)
ax.plot(distances, E_ENT)
ax.plot(distances, E_HEEM)
ax.plot(distances, E_EXACT, color='black', linestyle='--')

ax.legend(['$TPB$', '$ENT$', '$HEEM$'])

plt.savefig("Figures/E vs d.pdf")